In [103]:
import pandas as pd

ШАГ 1: ЗАГРУЗКА ДАННЫХ

In [104]:
workers = pd.read_csv("workers.csv")
equipment = pd.read_csv("equipment.csv")
schedule = pd.read_csv("schedule_template.csv")
requirements = pd.read_csv("position_requirements.csv")
plan = pd.read_csv("plan.csv")

print("workers - workers.csv")
display(workers.head())
print("equipment - equipment.csv")
display(equipment.head())
print("schedule - schedule_template.csv")
display(schedule.head())
print("requirements - position_requirements.csv")
display(requirements.head())
print("plan - plan.csv")
display(plan.head())

workers - workers.csv


,worker_id,name,плоская,высокая,струйная
0,W001,Работник П 01,7,6,5
1,W002,Работник П 02,6,5,0
2,W003,Работник П 03,5,4,0
3,W004,Работник П 04,4,0,5
4,W005,Работник П 05,7,6,0


equipment - equipment.csv


,machine_id,machine_type,operators_needed
0,PM-01,плоская,4
1,PM-02,плоская,4
2,PM-03,высокая,4
3,PM-04,высокая,4
4,SM-01,струйная,1


schedule - schedule_template.csv


,week,shift,worker_id
0,1,Ночь,W001
1,1,Ночь,W002
2,1,Ночь,W003
3,1,Ночь,W004
4,1,Ночь,W005


requirements - position_requirements.csv


,machine_type,position,min_rank,profession_required
0,плоская,1,7,Печатник
1,плоская,2,6,Печатник
2,плоская,3,5,Печатник
3,плоская,4,4,Печатник
4,струйная,1,5,Оператор струйной печати


plan - plan.csv


,machine_id,night,day,evening
0,PM-01,True,True,True
1,PM-02,True,True,True
2,PM-03,True,True,True
3,PM-04,True,True,True
4,SM-01,True,True,True


In [105]:
# Определяем основную профессию
cols = ["плоская", "высокая", "струйная"]
workers["основная_профессия"] = workers[cols].idxmax(axis=1)

# Добавляем все професии работника
workers["все_профессии"] = workers.apply(
    lambda row: [c for c in cols if row[c] > 0], axis=1
)

workers.head()

,worker_id,name,плоская,высокая,струйная,основная_профессия,все_профессии
0,W001,Работник П 01,7,6,5,плоская,"[плоская, высокая, струйная]"
1,W002,Работник П 02,6,5,0,плоская,"[плоская, высокая]"
2,W003,Работник П 03,5,4,0,плоская,"[плоская, высокая]"
3,W004,Работник П 04,4,0,5,струйная,"[плоская, струйная]"
4,W005,Работник П 05,7,6,0,плоская,"[плоская, высокая]"


In [106]:
# Параметры: генерируем для недели 1, понедельника, дневной смены
target_week = 2
# target_day = "Понедельник"
# target_shift = "День"
# shift = {"Ночь": "День", "День": "Вечер", "Вечер": "Ночь"}

In [107]:
# Старая вспомогательная функция shift_next_week удалена.
# Используйте build_shift_rotation ниже для расчёта ротации смен.

In [108]:
# Ранее здесь выполнялась проверка shift_next_week.
# Текущая версия рассчитывает кандидатов только через build_shift_rotation.

In [109]:
def build_shift_rotation(
    schedule: pd.DataFrame, workers: pd.DataFrame, target_week: int
) -> pd.DataFrame:
    """
    Формирует общий датафрейм кандидатов на target_week для всех смен сразу,
    применяя правило переворота смен:
        Ночь -> День, День -> Вечер, Вечер -> Ночь
    """
    shift_map = {"Ночь": "Вечер", "День": "Ночь", "Вечер": "День"}

    # Базовый слой — прошлая неделя
    prev = schedule.loc[schedule["week"] == target_week - 1].copy()

    # Сохраним прошлую смену (на всякий случай для анализа)
    prev = prev.rename(columns={"shift": "prev_shift"})

    # Рассчитаем смену на следующую неделю и проставим неделю
    prev["shift"] = prev["prev_shift"].map(shift_map)
    prev["week"] = target_week

    # Объединим с данными по работникам
    result = prev.merge(workers, on="worker_id", how="left")

    # Короткая статистика по сменам
    counts = result["shift"].value_counts()
    total = int(counts.sum())
    print(f"Кандидаты на работу в неделю {target_week}: всего {total}")
    for s in ["День", "Вечер", "Ночь"]:
        print(f"  {s}: {int(counts.get(s, 0))}")

    return result

In [110]:
shift_workers_all = build_shift_rotation(schedule, workers, target_week)
display(shift_workers_all.head())

# # При необходимости — получить конкретную смену:
# shift_workers_day   = shift_workers_all.query('shift == "День"')
# shift_workers_even  = shift_workers_all.query('shift == "Вечер"')
# shift_workers_night = shift_workers_all.query('shift == "Ночь"')

Кандидаты на работу в неделю 2: всего 60
  День: 20
  Вечер: 20
  Ночь: 20


,week,prev_shift,worker_id,shift,name,плоская,высокая,струйная,основная_профессия,все_профессии
0,2,Ночь,W001,Вечер,Работник П 01,7,6,5,плоская,"[плоская, высокая, струйная]"
1,2,Ночь,W002,Вечер,Работник П 02,6,5,0,плоская,"[плоская, высокая]"
2,2,Ночь,W003,Вечер,Работник П 03,5,4,0,плоская,"[плоская, высокая]"
3,2,Ночь,W004,Вечер,Работник П 04,4,0,5,струйная,"[плоская, струйная]"
4,2,Ночь,W005,Вечер,Работник П 05,7,6,0,плоская,"[плоская, высокая]"


In [113]:
# Для целевой смены получаем оборудование и требования по работникам
def f_shift_equipment(plan, shif_name):
    shift_equipment = plan[plan["shift"] == shif_name][
        ["week", "shift", "machine_id", "machine_type"]
    ].merge(requirements, on="machine_type", how="left")
    shift_equipment["worker_id"] = None

    return shift_equipment

In [114]:
# Преобразуем в длинный формат
plan_long = plan.melt(
    id_vars="machine_id",
    value_vars=["night", "day", "evening"],
    var_name="shift",
    value_name="works",
)

# Оставляем только те строки, где машина работает
plan_long = (
    plan_long[plan_long["works"] == True].drop(columns="works").reset_index(drop=True)
)

# Заменяем названия смен на русские
plan_long = plan_long.replace(
    {"shift": {"night": "Ночь", "day": "День", "evening": "Вечер"}}
)

plan_long = plan_long.merge(
    equipment[["machine_id", "machine_type"]], on="machine_id", how="left"
)
plan_long["week"] = target_week


# Создаем расписание для каждой смены с пустыми позициями работников
shift_equipment_day = f_shift_equipment(plan_long, "День")
shift_equipment_evening = f_shift_equipment(plan_long, "Вечер")
shift_equipment_night = f_shift_equipment(plan_long, "Ночь")


display(shift_equipment_day.head())
display(shift_equipment_night.head())
display(shift_equipment_evening.head())

,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,День,PM-01,плоская,1,7,Печатник,None
1,2,День,PM-01,плоская,2,6,Печатник,None
2,2,День,PM-01,плоская,3,5,Печатник,None
3,2,День,PM-01,плоская,4,4,Печатник,None
4,2,День,PM-02,плоская,1,7,Печатник,None


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Ночь,PM-01,плоская,1,7,Печатник,None
1,2,Ночь,PM-01,плоская,2,6,Печатник,None
2,2,Ночь,PM-01,плоская,3,5,Печатник,None
3,2,Ночь,PM-01,плоская,4,4,Печатник,None
4,2,Ночь,PM-02,плоская,1,7,Печатник,None


,week,shift,machine_id,machine_type,position,min_rank,profession_required,worker_id
0,2,Вечер,PM-01,плоская,1,7,Печатник,None
1,2,Вечер,PM-01,плоская,2,6,Печатник,None
2,2,Вечер,PM-01,плоская,3,5,Печатник,None
3,2,Вечер,PM-01,плоская,4,4,Печатник,None
4,2,Вечер,PM-02,плоская,1,7,Печатник,None


In [115]:
def candidat(
    shift_workers_all,
    assigned_shift,
    global_assigned,
    mode,
    profession,
    min_rank,
    shift_name,
):
    assigned_shift = set(assigned_shift)
    global_assigned = set(global_assigned or set())
    blocked_ids = global_assigned - assigned_shift

    base_mask = (
        (shift_workers_all["shift"] == shift_name)
        & (~shift_workers_all["worker_id"].isin(blocked_ids))
        & (~shift_workers_all["worker_id"].isin(assigned_shift))
    )

    profession_mask = shift_workers_all["все_профессии"].apply(
        lambda profs: profession in profs
    )

    if mode == "ferst":
        primary_mask = shift_workers_all["основная_профессия"] == profession
        rank_mask = shift_workers_all[profession] == min_rank
        candidates = shift_workers_all[base_mask & primary_mask & rank_mask]
    elif mode == "second":
        rank_mask = shift_workers_all[profession].isin([min_rank, min_rank + 1])
        candidates = shift_workers_all[base_mask & profession_mask & rank_mask]
    elif mode == "third":
        rank_mask = shift_workers_all[profession] > 0
        candidates = shift_workers_all[base_mask & profession_mask & rank_mask]
    else:
        raise ValueError(
            f"Неизвестный режим '{mode}'. Используйте 'ferst', 'second' или 'third'."
        )

    return candidates.sort_values(by=[profession, "worker_id"], ascending=[False, True])


def position_assignment(
    shift_equipment,
    shift_workers_all,
    assigned_shift,
    global_assigned,
    mode="ferst",
    shift_name="День",
):
    free_positions = []
    updated = shift_equipment.copy()

    for index, row in updated.iterrows():
        worker_id = row.get("worker_id")
        if pd.isna(worker_id) or worker_id in ("", None):
            profession = row["machine_type"]
            min_rank = row["min_rank"]
            candidates = candidat(
                shift_workers_all,
                assigned_shift,
                global_assigned,
                mode,
                profession,
                min_rank,
                shift_name,
            )

            if not candidates.empty:
                chosen = candidates.iloc[0]
                updated.loc[index, "worker_id"] = chosen["worker_id"]
                assigned_shift.add(chosen["worker_id"])
            else:
                free_positions.append(updated.loc[index])

    columns = updated.columns
    free_df = (
        pd.DataFrame(free_positions, columns=columns)
        if free_positions
        else pd.DataFrame(columns=columns)
    )

    return free_df, updated, assigned_shift

In [116]:
def run_assignment_for_shift(
    shift_equipment,
    shift_workers_all,
    default_rounds,
    global_assigned,
):
    assigned_shift = set()

    free_positions, updated, assigned_shift = position_assignment(
        shift_equipment,
        shift_workers_all,
        assigned_shift,
        global_assigned,
        mode=default_rounds[0][0],
        shift_name=default_rounds[0][1],
    )

    for round_idx, (mode, shift_name) in enumerate(default_rounds[1:], start=2):
        if free_positions.empty:
            break
        print(
            f"Остались свободные позиции после тура {round_idx - 1}: {len(free_positions)}"
        )
        free_positions, patch, assigned_shift = position_assignment(
            free_positions,
            shift_workers_all,
            assigned_shift,
            global_assigned,
            mode=mode,
            shift_name=shift_name,
        )
        updated = updated.combine_first(patch)

    return updated, assigned_shift

In [117]:
def summary_team(shift_equipment):
    summary = shift_equipment.groupby(
        ["machine_id", "machine_type"], as_index=False
    ).agg(
        required=("position", "count"),
        assigned=("worker_id", lambda s: s.notna().sum() - (s == "").sum()),
    )
    return summary


def incomplete_team(shift_equipment):
    summary = summary_team(shift_equipment)
    incomplete = summary[
        (summary["assigned"] > 0) & (summary["assigned"] < summary["required"])
    ].copy()
    return incomplete


def decomlate_team(shift_equipment, assigned_shift):
    incomplete = incomplete_team(shift_equipment)
    destaff = incomplete[incomplete["required"] / 2 >= incomplete["assigned"]][
        "machine_id"
    ].to_list()

    if not destaff:
        return shift_equipment, assigned_shift

    mask = (
        shift_equipment["machine_id"].isin(destaff)
        & shift_equipment["worker_id"].notna()
    )
    freed = shift_equipment.loc[mask, "worker_id"].dropna().tolist()

    shift_equipment.loc[mask, "worker_id"] = None
    assigned_shift -= set(freed)

    print(f"Расформированы бригады: {destaff}")
    return shift_equipment, assigned_shift

In [118]:
def staff_team(
    shift_equipment,
    shift_workers_all,
    assigned_shift,
    global_assigned,
    shift_name="Ночь",
    mode="third",
):
    incomplete = incomplete_team(shift_equipment)
    if incomplete.empty:
        return shift_equipment, assigned_shift

    mask = shift_equipment["machine_id"].isin(incomplete["machine_id"])
    free_positions, patch, assigned_shift = position_assignment(
        shift_equipment.loc[mask].copy(),
        shift_workers_all,
        assigned_shift,
        global_assigned,
        mode=mode,
        shift_name=shift_name,
    )

    updated = shift_equipment.copy()
    if not patch.empty:
        updated.update(patch[["worker_id"]])

    return updated, assigned_shift

In [119]:
# Назначение работников на позиции
default_tourse = [
    ("ferst", "День"),
    ("second", "День"),
    ("ferst", "Ночь"),
    ("second", "Ночь"),
    ("ferst", "Вечер"),
    ("second", "Вечер"),
]

default_tourse_day = default_tourse.copy()
default_tourse_evening = default_tourse[4:] + default_tourse[:4]
default_tourse_night = default_tourse[2:] + default_tourse[:2]

global_assigned = set()

shift_equipment_day, assigned_day = run_assignment_for_shift(
    shift_equipment_day,
    shift_workers_all,
    default_tourse_day,
    global_assigned,
)
shift_equipment_day, assigned_day = decomlate_team(shift_equipment_day, assigned_day)
shift_equipment_day, assigned_day = staff_team(
    shift_equipment_day,
    shift_workers_all,
    assigned_day,
    global_assigned,
    shift_name="День",
)
global_assigned.update(assigned_day)

shift_equipment_evening, assigned_evening = run_assignment_for_shift(
    shift_equipment_evening,
    shift_workers_all,
    default_tourse_evening,
    global_assigned,
)
shift_equipment_evening, assigned_evening = decomlate_team(
    shift_equipment_evening, assigned_evening
)
shift_equipment_evening, assigned_evening = staff_team(
    shift_equipment_evening,
    shift_workers_all,
    assigned_evening,
    global_assigned,
    shift_name="Вечер",
)
global_assigned.update(assigned_evening)

shift_equipment_night, assigned_night = run_assignment_for_shift(
    shift_equipment_night,
    shift_workers_all,
    default_tourse_night,
    global_assigned,
)
shift_equipment_night, assigned_night = decomlate_team(
    shift_equipment_night, assigned_night
)
shift_equipment_night, assigned_night = staff_team(
    shift_equipment_night,
    shift_workers_all,
    assigned_night,
    global_assigned,
    shift_name="Ночь",
)
global_assigned.update(assigned_night)

Остались свободные позиции после тура 1: 21
Остались свободные позиции после тура 2: 17
Остались свободные позиции после тура 3: 13
Остались свободные позиции после тура 4: 9
Остались свободные позиции после тура 5: 5
Остались свободные позиции после тура 1: 9
Остались свободные позиции после тура 2: 9
Остались свободные позиции после тура 3: 9
Остались свободные позиции после тура 4: 9
Остались свободные позиции после тура 5: 8
Остались свободные позиции после тура 1: 13
Остались свободные позиции после тура 2: 13
Остались свободные позиции после тура 3: 13
Остались свободные позиции после тура 4: 13
Остались свободные позиции после тура 5: 13
Расформированы бригады: ['PM-01']


In [120]:
assigned_sets = {
    "День": assigned_day,
    "Вечер": assigned_evening,
    "Ночь": assigned_night,
}

shift_tables = {
    "День": shift_equipment_day,
    "Вечер": shift_equipment_evening,
    "Ночь": shift_equipment_night,
}

shift_gap_report = []
for shift_name, table in shift_tables.items():
    open_positions = int(table["worker_id"].isna().sum())
    shift_gap_report.append(
        {
            "shift": shift_name,
            "assigned_workers": len(assigned_sets[shift_name]),
            "open_positions": open_positions,
        }
    )
shift_gap_report = pd.DataFrame(shift_gap_report)
print(shift_gap_report.to_string(index=False))


def remaining_candidates(shift_workers_all, assigned_sets):
    rows = []
    for shift_name, assigned in assigned_sets.items():
        pool = shift_workers_all[shift_workers_all["shift"] == shift_name]
        free = pool[~pool["worker_id"].isin(assigned)]
        rows.append(
            {
                "shift": shift_name,
                "available_workers": int(len(free)),
            }
        )
    return pd.DataFrame(rows)


free_pool_report = remaining_candidates(shift_workers_all, assigned_sets)
print(free_pool_report.to_string(index=False))

unfilled_positions = pd.concat(
    [
        shift_equipment_day[shift_equipment_day["worker_id"].isna()],
        shift_equipment_evening[shift_equipment_evening["worker_id"].isna()],
        shift_equipment_night[shift_equipment_night["worker_id"].isna()],
    ],
    ignore_index=True,
)
print(f"Незаполненные позиции: {len(unfilled_positions)}")

shift  assigned_workers  open_positions
 День                37               0
Вечер                16               4
 Ночь                 7              13
shift  available_workers
 День                  0
Вечер                  9
 Ночь                 13
Незаполненные позиции: 17


In [98]:
# Собираем все смены в один график Ночь, День, Вечер
shift_equipment = pd.concat(
    [shift_equipment_night, shift_equipment_day, shift_equipment_evening],
    ignore_index=True,
)

In [ ]:
result = shift_equipment.merge(
    workers[["worker_id", "name"]],
    on="worker_id",
    how="left",
)

assigned_rows = result[result["worker_id"].notna()].copy()
assigned_rows = assigned_rows.sort_values(
    by=["shift", "machine_id", "position"],
    ignore_index=True,
)

assigned_rows.to_csv("assignment_output.csv", index=False, encoding="utf-8-sig")
print("assignment_output.csv обновлен")

schedule_wo_target = schedule[schedule["week"] != target_week].copy()
updated_schedule = pd.concat(
    [
        schedule_wo_target,
        assigned_rows[["week", "shift", "worker_id"]].drop_duplicates(),
    ],
    ignore_index=True,
)
updated_schedule.to_csv("schedule_template.csv", index=False, encoding="utf-8-sig")
print("schedule_template.csv обновлен")

In [99]:
brigade_summary = summary_team(shift_equipment)
print(brigade_summary.to_string(index=False))

machine_id machine_type  required  assigned
     PM-01      плоская        12         8
     PM-02      плоская        12         8
     PM-03      высокая        12        12
     PM-04      высокая        12        11
     SM-01     струйная         3         1
     SM-02     струйная         3         1
     SM-03     струйная         3         1
     SM-04     струйная         3         1
     SM-05     струйная         1         1
     SM-06     струйная         1         1
     SM-07     струйная         1         1
     SM-08     струйная         1         1
     SM-09     струйная         1         1
     SM-10     струйная         1         1
     SM-11     струйная         1         1
     SM-12     струйная         1         1
     SM-13     струйная         1         1
     SM-14     струйная         1         1
     SM-15     струйная         1         1
     SM-16     струйная         1         1
     SM-17     струйная         1         1
     SM-18     струйная         

In [100]:
updated_schedule.head()

,week,shift,worker_id
0,1,Ночь,W001
1,1,Ночь,W002
2,1,Ночь,W003
3,1,Ночь,W004
4,1,Ночь,W005


In [101]:
assigned_rows[["week", "shift", "machine_id", "position", "worker_id", "name"]]

,week,shift,machine_id,position,worker_id,name
0,2,Вечер,PM-01,1,W005,Работник П 05
1,2,Вечер,PM-01,2,W002,Работник П 02
2,2,Вечер,PM-01,3,W003,Работник П 03
3,2,Вечер,PM-01,4,W043,Работник П 43
4,2,Вечер,PM-02,1,W041,Работник П 41
5,2,Вечер,PM-02,2,W006,Работник П 06
6,2,Вечер,PM-02,3,W007,Работник П 07
7,2,Вечер,PM-02,4,W047,Работник П 47
8,2,Вечер,PM-03,1,W009,Работник В 09
9,2,Вечер,PM-03,2,W010,Работник В 10
